In [1]:
# set root directory of data
DATA = f"/home/gologors/data/positive_pituitary"

In [5]:
# utilities
import os, sys, time, json, glob, pprint

# segmentation
import meshio
from scipy.spatial   import Delaunay
    
# graphing
from matplotlib import gridspec

import matplotlib.pyplot as plt
import seaborn as sns

# data
import numpy as np
import pandas as pd

# nii
import SimpleITK as sitk

# interactive
from ipywidgets import interact, interactive, IntSlider, ToggleButtons, fixed

# helpers

# add root to filepath (=os.path.join)
def getfp(fn, root=DATA): return f"{root}/{fn}"

# sitk obj and np array have different index conventions
# deep copy
def sitk2np(obj): return np.swapaxes(sitk.GetArrayFromImage(obj), 0, 2)
def np2sitk(arr): return sitk.GetImageFromArray(np.swapaxes(arr, 0, 2))

# round all floats in a tuple to 3 decimal places
def round_tuple(t, d=2): return tuple(round(x,d) for x in t)

def orient_test(image):
    orient = sitk.DICOMOrientImageFilter()
    orient.DebugOn()
    print(round_tuple(image.GetDirection(), d=2))
    print(orient.GetOrientationFromDirectionCosines(image.GetDirection()))

# print sitk info
def print_sitk_info(image):   
    orient = sitk.DICOMOrientImageFilter()

    print("Size: ", image.GetSize())
    print("Origin: ", image.GetOrigin())
    print("Spacing: ", image.GetSpacing())
    print("Direction: ", round_tuple(image.GetDirection(), d=2))
    print("Orientation: ", orient.GetOrientationFromDirectionCosines(image.GetDirection()))
    print(f"Pixel type: {image.GetPixelIDValue()} = {image.GetPixelIDTypeAsString()}")

In [3]:
# read sequence names from csv
folders = os.listdir(DATA)
print(len(folders))

# folder
folder = folders[0]

10


In [4]:
# Select sequence
seq_dict = {i:i for i in range(len(folders))}
select_seq = ToggleButtons(
    options=seq_dict.keys(),
    description='Select Seq:',
    disabled=False,
    button_style='info', 
    
)

#IntSlider(min=0, max=len(series_info)-1, description='Select Seq', continuous_update=False)

# Create button values
axis_dict = {'Axis 0': 0, 'Axis 1': 1, 'Axis 2': 2}
select_axis = ToggleButtons(
    options=['Axis 0','Axis 1', 'Axis 2'],
    description='Select Axis:',
    disabled=False,
    button_style='info', 
    
)

# Select layer
nii_num_dict = {
    "#1": 0,
    "#2": 1,
    "#3": 2
}
select_nii = ToggleButtons(
    options=nii_num_dict.keys(),
    description='Select nii:',
    disabled=False,
    button_style='info', 
    
)

# Select layer
select_layer = IntSlider(min=0, max=20, description='Select Layer', continuous_update=False)
    
# Define a function for plotting images
def plot_image(seq, axis, nii_num, layer, reorient):
    
    # Get axis, slice
    seq_idx = seq_dict[seq]
    axis = axis_dict[axis]
    
    # get nii num
    nii_idx = nii_num_dict[nii_num]
        
    # get sequence
    fn  = folders[seq]

    # get files in dir
    all_files = sorted(os.listdir(getfp(fn)))
    niis  = [x for x in all_files if x.endswith(".nii.gz")]
    jsons = [x for x in all_files if x.endswith(".json")] 
    dcms  = [x for x in all_files if x.endswith(".dcm")]

    select_nii.options = list(nii_num_dict.keys())[:len(niis)]
    
    # open .nii files
    im_obj = sitk.ReadImage(getfp(f"{fn}/{niis[nii_idx]}"))
    
    # reorient to LAS
    if reorient:
        # old orientation
        old_orient = sitk.DICOMOrientImageFilter().GetOrientationFromDirectionCosines(im_obj.GetDirection())
        im_obj = sitk.DICOMOrient(im_obj, "LAS")
        
    im_arr = sitk2np(im_obj)
    im = np.take(im_arr, layer, axis=axis)

    # print info           
    print(f"Seq: {seq}")
    print("Num dcms: ", len(dcms))
    print("Num niis: ", len(niis))
    if reorient:
        print("Old orientation: ", old_orient)
    print_sitk_info(im_obj)
    print(f"Plotting slice idx {layer} / {im_arr.shape[axis]}")

    select_layer.max = im_arr.shape[axis]-1
    #select_layer.value = select_layer.max//2

    # plot
    plt.figure(figsize=(10,5))
    plt.imshow(np.rot90(im), cmap='gray')
    plt.axis('off');
    plt.show()
    return seq, axis, layer

# Use the interactive() tool to create the visualization
interactive(plot_image, seq=select_seq, layer=select_layer, axis=select_axis, nii_num=select_nii, reorient=False)

interactive(children=(ToggleButtons(button_style='info', description='Select Seq:', options=(0, 1, 2, 3, 4, 5,…

# View annotations

- dictionary: 

Dict: nii, .mtl

In [5]:
def get_suffix(x, suffix):
    return glob.glob(f"{getfp(x)}/*.{suffix}")

In [6]:
# for each folder, get path to MR (.nii) and mesh (.obj)
mesh_dict = {}
for x in os.listdir(DATA):
    niis = get_suffix(x, "nii.gz") + get_suffix(x, "nii")
    meshes = get_suffix(x, "obj")
    
    if len(niis) == 1 and len(meshes) == 1:
        mesh_dict[x] = (niis[0], meshes[0])
    else:
        print(f"Not unique: {x} has {len(niis)} niis and {len(meshs)} meshes.")

# print(json.dumps(nii_mtl_dict, indent=2))

In [7]:
# plot

from helpers.viz import viz_axis, viz_compare_inputs, viz_compare_outputs
from helpers.viz import *

def viz_anno(mr, seg):
    
    mr, seg = np.array(mr), np.array(seg)
    gt_bbox = mask2bbox(seg)
    
    bbox = gt_bbox
    
    # print bbox
    #print("Pred: "); print_bbox(*pred_bbox)
    print("GT: "); print_bbox(*gt_bbox)
          
    # viz
    viz_axis(np_arr = mr, \
            bin_mask_arr   = seg,     color1 = "yellow",  alpha1=0.3, \
            #bin_mask_arr2  = pred_mk, color2 = "magenta", alpha2=0.3, \
            slices=lrange(*bbox[4:6]), fixed_axis=2, \
            axis_fn = np.rot90, \
            title   = "Axis 2", \

#             np_arr_b = mr, \
#             bin_mask_arr_b   = seg,     color1_b = "yellow",  alpha1_b=0.3, \
#             bin_mask_arr2_b  = pred_mk, color2_b = "magenta", alpha2_b=0.3, \
#             slices_b = lrange(*bbox[2:4]), fixed_axis_b=1, \
#             title_b  = "Axis 1", \

#             np_arr_c = mr, \
#             bin_mask_arr_c   = seg,     color1_c = "yellow",  alpha1_c=0.3, \
#             bin_mask_arr2_c  = pred_mk, color2_c = "magenta", alpha2_c=0.3, \
#             slices_c = lrange(*bbox[4:6]), fixed_axis_c=2, \
#             title_c = "Axis 2", \
  
        ncols = 5, hspace=0.3, fig_mult=2) 

In [8]:
# Convert segmentation object to numpy binary mask
# 1. Get affine matrix in SITK (aff tfm: idx coord => physical space coord)
# 2. Convert image idxs to physical coords
# 3. Check whether physical coords are in the Delauney triangulation of segmented mesh points

# numpy mask arr into sitk obj
def mask2sitk(mask_arr, sitk_image):
  # convert bool mask to int mask
  # swap axes for sitk
  obj = sitk.GetImageFromArray(np.swapaxes(mask_arr.astype(np.uint8), 0, 2))
  obj.SetOrigin(sitk_image.GetOrigin())
  obj.SetSpacing(sitk_image.GetSpacing())   
  obj.SetDirection(sitk_image.GetDirection())
  return obj

# 1. Get affine matrix in SITK
# https://niftynet.readthedocs.io/en/v0.2.2/_modules/niftynet/io/simple_itk_as_nibabel.html
def make_affine(simpleITKImage, ras_adj):
    # get affine transform in LPS
    c = [simpleITKImage.TransformIndexToPhysicalPoint(p)
         for p in ((1, 0, 0),
                   (0, 1, 0),
                   (0, 0, 1),
                   (0, 0, 0))]
    c = np.array(c)
    affine = np.concatenate([
        np.concatenate([c[0:3] - c[3:], c[3:]], axis=0),
        [[0.], [0.], [0.], [1.]]
    ], axis=1)
    affine = np.transpose(affine)
    # convert to RAS to match nibabel
    if ras_adj:
        affine = np.matmul(np.diag([-1., -1., 1., 1.]), affine)
    return affine

# given paths, return isotropic SITK obj of nii and segm obj
def paths2objs(mr_path, segm_path, ras_adj = False):
    mr         = sitk.ReadImage(mr_path, sitk.sitkFloat32)
    segm       = meshio.read(segm_path)
    mask_arr   = seg2mask(mr, segm, ras_adj)
    
    return mr, mask2sitk(mask_arr, mr)

# Seg2mask
def seg2mask(image_obj, segm_obj, ras_adj):
    dims = image_obj.GetSize()
    aff     = make_affine(image_obj, ras_adj)
    idx_pts = np.indices(dims[::-1], dtype=np.uint16).T.reshape(-1,3)[:,[2,1,0]]
    physical_pts = (np.dot(aff[:3,:3], idx_pts.T) + aff[:3,3:4]).T 
    return (Delaunay(segm_obj.points).find_simplex(physical_pts) >= 0).reshape(dims)


In [9]:
# try one
x = list(mesh_dict.keys())[9]
print(x)

1.3.46.670589.11.37169.5.0.3436.2016110300230654926


In [10]:
mesh_dict[x]

('/home/gologors/data/positive_pituitary/1.3.46.670589.11.37169.5.0.3436.2016110300230654926/1.3.46.670589.11.37169.5.0.3436.2016110300230654926_COR_T1_CLEAR_20161103000404_701.nii.gz',
 '/home/gologors/data/positive_pituitary/1.3.46.670589.11.37169.5.0.3436.2016110300230654926/Segmentation.obj')

In [11]:
mr_path, segm_path = mesh_dict[x]

mr         = sitk.ReadImage(mr_path, sitk.sitkFloat32)
segm       = meshio.read(segm_path)

In [12]:
image_obj = mr
segm_obj = segm

ras_adj = False
dims = image_obj.GetSize()
aff     = make_affine(image_obj, ras_adj)
idx_pts = np.indices(dims[::-1], dtype=np.uint16).T.reshape(-1,3)[:,[2,1,0]]
physical_pts = (np.dot(aff[:3,:3], idx_pts.T) + aff[:3,3:4]).T 

In [13]:
blokc

NameError: name 'blokc' is not defined

In [14]:
list(mesh_dict.keys())[7]

'1.3.46.670589.11.70769.5.0.6828.2016112311204114626'

In [15]:
list(mesh_dict.keys())[9]

'1.3.46.670589.11.37169.5.0.3436.2016110300230654926'

In [ ]:
x1 = Delaunay(segm_obj.points)

In [ ]:
bloack

In [ ]:
x2 = x1.find_simplex(physical_pts)

In [ ]:
res = (Delaunay(segm_obj.points).find_simplex(physical_pts) >= 0).reshape(dims)

In [ ]:
mr_obj, seg_obj = paths2objs(*mesh_dict[x])

In [ ]:
mr = sitk2np(mr_obj)
seg = sitk2np(seg_obj)

viz_anno(mr, seg)
# viz_compare_outputs(mr, seg, seg)

In [ ]:
# make a dictionary of key = train folder, value = (segm obj, nii file)
def get_data_dict_n4(train_path):
    train_folders   = os.listdir(train_path)
    train_data_dict = {}
    for folder in train_folders:
        segm_obj_path = os.path.join(train_path, folder, "seg.pt")

        mp_path      = os.path.join(train_path, folder, "MP-RAGE")
        folder1_path = os.path.join(mp_path, os.listdir(mp_path)[0])
        folder2_path = os.path.join(folder1_path, os.listdir(folder1_path)[0])

        # choose corrected_n4 if available
        nii_paths = glob.glob(f"{folder2_path}/*.nii")
        nii_path = nii_paths[0]
         
        if len(nii_paths) > 1 and not nii_path.endswith("corrected_n4.nii"):
            nii_path = nii_paths[1]
            
        train_data_dict[folder] = (nii_path, segm_obj_path) #(segm_obj_path, nii_path)
    return train_data_dict